In [1]:
import numpy as np
import random

class Structure:
    def __init__(self, seed, seed_next_possible, portion, destination):
        self.history = None
        self.data = None
        self.rect = []
        self.next_possibles = None
        
        self.portion = portion
        self.seed = seed
        self.seed_next_possible = seed_next_possible
        
        self.cleanUp()
        self.destination = destination
        self.preprocess()

        #print("initialized data: \n", self.data)
    
    def preprocess(self):
        start = self.seed_next_possible[0]
        direction = start[-1]
        new_vertex_clean = np.array([start[0], start[1], start[2]])
        self.add_vertex(new_vertex_clean)
        
        span = 4 + int(random.random() * 6 % 6)
        for i in range(span):
            new_vertex_clean = self.get_vertex_forward(new_vertex_clean, direction)
            self.add_vertex(new_vertex_clean)
        
        self.remove_possible(0)
        self.history = np.append(self.history, self.data[1:], axis=0)
        self.rect.append(block_to_rect(self.data, self.portion))
        self.data = np.array([[0,0,0]])
        
        self.to_dest(new_vertex_clean)


    def to_dest(self, new_vertex_clean):
        x_dist = abs(self.destination[0] - new_vertex_clean[0])
        x_start = min(self.destination[0], new_vertex_clean[0])
        startPos = np.array([x_start, new_vertex_clean[1], new_vertex_clean[2]])
        scale = np.array([x_dist+0.07*self.portion, 0.07*self.portion, 0.07*self.portion])
        rect1 = rect(startPos, scale)
        
        y_dist = abs(self.destination[1] - new_vertex_clean[1])
        y_start = min(self.destination[1], new_vertex_clean[1])
        startPos2 = np.array([self.destination[0], y_start, new_vertex_clean[2]])
        scale2 = np.array([0.07,y_dist+0.07*self.portion, 0.07*self.portion])
        rect2 = rect(startPos2, scale2)
        
        self.rect.append(rect1)
        self.rect.append(rect2)
        
        
    def cleanUp(self):
        self.history = self.seed
        self.data = np.array([[0.0,0.0,0.0]])
        self.data = np.append(self.data, self.seed, axis = 0)
        self.rect = []
        self.rect.append(block_to_rect(self.data, self.portion))
        self.data = np.array([[0.0,0.0,0.0]])
        self.next_possibles = self.seed_next_possible

  
    def cost_func(self, pos):
        return random.random()
  
    def add_vertex(self, next_vertex):
        x = next_vertex[0]
        y = next_vertex[1]
        z = next_vertex[2]

        self.data = np.vstack([self.data, np.array([x,y,z])])

    def remove_possible(self, next_index):
        self.next_possibles = np.delete(self.next_possibles, next_index, axis=0)
  
  #define direction:
  #x=-1 -> 0, x=1 -> 1, y=-1 -> 2, y=1 -> 3, z=-1 -> 4, z=1 -> 5,
    def get_next_possible(self, available_contact, direction):
        result = np.array([0,0,0,0])

        available_contact = np.append(available_contact, 0)
        if direction == 0 or direction == 1:
            result = np.vstack([result, available_contact + np.array([0.0,0.07,0.0,3])])
            result = np.vstack([result, available_contact + np.array([0.0,-0.07,0.0,2])])
            result = np.delete(result, 0, axis =0)
            return result

        if direction == 2 or direction == 3:
            result = np.vstack([result, available_contact + np.array([0.07,0.0,0.0,1])])
            result = np.vstack([result, available_contact + np.array([-0.07,0.0,0.0,1])])
            result = np.delete(result, 0, axis =0)
            return result

    
    def get_last_direction(self,direction):
        count =direction[1]*1 + direction[2]*2
        return count
  
    def get_vertex_forward(self, new_vertex_clean, direction):
        if direction == 0:
            return new_vertex_clean + np.array([-0.07,0.0,0.0])

        if direction == 1:
            return new_vertex_clean + np.array([0.07,0.0,0.0])

        if direction == 2:
            return new_vertex_clean + np.array([0.0,-0.07,0.0])

        if direction == 3:
            return new_vertex_clean + np.array([0.0,0.07,0.0])
    
    def generate(self, steps):
        for i in range(steps):
            #print("step ", i+1)
            self.process()
            self.history = np.append(self.history, self.data[1:], axis=0)
            self.rect.append(block_to_rect(self.data, self.portion))
            self.data = np.array([[0,0,0]])



    def process(self):
        #step 1: for all available contacts, assume they will make a turn  
        #print("next_possibles", self.next_possibles)

        #step2: create a 1d array that stores cost for each possible point
        costs = np.array([])
        for next_possible in self.next_possibles:
            costs = np.append(costs, self.cost_func(next_possible))

        #print("costs", costs)

        #step3 find the next_vertex with smallest cost
        next_index = np.argpartition(costs, -1)[-1:]
        next_vertex = self.next_possibles[next_index] 

        #print("next_vertex", next_vertex)

        #step4 add next_vertex to data
        new_direction = next_vertex[0][3]
        #print("new_direction", new_direction)
        new_vertex_clean = np.array([next_vertex[0][0], next_vertex[0][1], next_vertex[0][2]])
        #print("new_vertex_clean", new_vertex_clean)
        self.add_vertex(new_vertex_clean)
        
        span = 3 + int(random.random() * 6 % 6)
        for i in range(span):
            new_vertex_clean = self.get_vertex_forward(new_vertex_clean, new_direction)
            self.add_vertex(new_vertex_clean)

        #print("data: \n", self.data)

        #step5 update next_possible
        self.remove_possible(next_index)
        #print("next_possibles, removed old", self.next_possibles)
        #print("next_vertex", new_vertex_clean)
        #print("new_direction", new_direction)
        new_next_possibles = self.get_next_possible(new_vertex_clean, new_direction)
        self.next_possibles = np.append(self.next_possibles,new_next_possibles, axis=0)
        #print("next_possibles, add new", self.next_possibles)


In [2]:
def block_to_rect(buffer, portion):
    start = buffer[1]
    end = buffer[-1]
    x_start = min(start[0],end[0])
    y_start = min(start[1],end[1])    
    z_start = min(start[2],end[2])
    
    x_scale = max(abs(start[0] - end[0]), 0.05 * portion)
    y_scale = max(abs(start[1] - end[1]), 0.05 * portion)
    z_scale = max(abs(start[2] - end[2]), 0.05 * portion)
    
    if(x_scale > 0.05 * portion):
        x_scale += 0.05 * portion
        
    if(y_scale > 0.05 * portion):
        y_scale += 0.05 * portion
        
    if(z_scale > 0.05 * portion):
        z_scale += 0.05 * portion

    startPos = np.array([x_start,y_start,z_start])
    scale = np.array([x_scale,y_scale,z_scale])
    
    tempt = rect(startPos, scale)
    return tempt


In [3]:
seed = np.array([[0.0,0.0,0.0], [0.0,0.1,0.0],[0.0,0.2,0.0],[0.0,0.3,0.0], [0.0,0.4,0.0]])
seed_next_possible = np.array([[-0.1,0.4,0.0,0],[0.1,0.4,0.0,1]])


In [4]:
class rect:
    def __init__(self, startPos, scale): 
        self.start_x = round(startPos[0],2)
        self.start_y = round(startPos[1],2)
        self.start_z = round(startPos[2],2)
        
        self.scale_x = round(scale[0],2)
        self.scale_y = round(scale[1],2)
        self.scale_z = round(scale[2],2)
    
    def info(self):
        print("start_x:",self.start_x,"start_y:",self.start_y,"start_z:",self.start_z)
        print("scale_x:",self.scale_x,"scale_y:",self.scale_y,"scale_z:",self.scale_z)
        print(" ")


In [5]:
sa = Structure(seed, seed_next_possible, 0.5, np.array([0.4,0.9]))

sa.generate(5)

In [6]:
sa.history

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.1 ,  0.  ],
       [ 0.  ,  0.2 ,  0.  ],
       [ 0.  ,  0.3 ,  0.  ],
       [ 0.  ,  0.4 ,  0.  ],
       [-0.1 ,  0.4 ,  0.  ],
       [-0.17,  0.4 ,  0.  ],
       [-0.24,  0.4 ,  0.  ],
       [-0.31,  0.4 ,  0.  ],
       [-0.38,  0.4 ,  0.  ],
       [-0.45,  0.4 ,  0.  ],
       [-0.52,  0.4 ,  0.  ],
       [-0.59,  0.4 ,  0.  ],
       [-0.66,  0.4 ,  0.  ],
       [-0.73,  0.4 ,  0.  ],
       [ 0.1 ,  0.4 ,  0.  ],
       [ 0.17,  0.4 ,  0.  ],
       [ 0.24,  0.4 ,  0.  ],
       [ 0.31,  0.4 ,  0.  ],
       [ 0.31,  0.47,  0.  ],
       [ 0.31,  0.54,  0.  ],
       [ 0.31,  0.61,  0.  ],
       [ 0.31,  0.68,  0.  ],
       [ 0.31,  0.75,  0.  ],
       [ 0.31,  0.82,  0.  ],
       [ 0.31,  0.89,  0.  ],
       [ 0.31,  0.96,  0.  ],
       [ 0.24,  0.96,  0.  ],
       [ 0.31,  0.96,  0.  ],
       [ 0.38,  0.96,  0.  ],
       [ 0.45,  0.96,  0.  ],
       [ 0.52,  0.96,  0.  ],
       [ 0.59,  0.96,  0.  ],
       [ 0

In [7]:
seed2 = np.array([[0.0,0.0,0.0], [0.1,0.0,0.0],[0.2,0.0,0.0],[0.3,0.0,0.0], [0.4,0.0,0.0]])
seed2_next_possible = np.array([[0.4,0.1,0.0,3],[0.4,-0.1,0.0,2]])
sa2 = Structure(seed2, seed2_next_possible, 0.5,np.array([0.4,0.9]))

sa2.generate(5)

In [8]:
sa.history = np.round(sa.history, 1)

In [9]:
for i in sa.rect:
    i.info()

start_x: 0.0 start_y: 0.0 start_z: 0.0
scale_x: 0.02 scale_y: 0.43 scale_z: 0.02
 
start_x: -0.73 start_y: 0.4 start_z: 0.0
scale_x: 0.66 scale_y: 0.02 scale_z: 0.02
 
start_x: -0.73 start_y: 0.4 start_z: 0.0
scale_x: 1.17 scale_y: 0.04 scale_z: 0.04
 
start_x: 0.4 start_y: 0.4 start_z: 0.0
scale_x: 0.07 scale_y: 0.54 scale_z: 0.04
 
start_x: 0.1 start_y: 0.4 start_z: 0.0
scale_x: 0.24 scale_y: 0.02 scale_z: 0.02
 
start_x: 0.31 start_y: 0.47 start_z: 0.0
scale_x: 0.02 scale_y: 0.52 scale_z: 0.02
 
start_x: 0.24 start_y: 0.96 start_z: 0.0
scale_x: 0.45 scale_y: 0.02 scale_z: 0.02
 
start_x: 0.31 start_y: -0.09 start_z: 0.0
scale_x: 0.02 scale_y: 0.45 scale_z: 0.02
 
start_x: 0.24 start_y: -0.09 start_z: 0.0
scale_x: 0.52 scale_y: 0.02 scale_z: 0.02
 


In [13]:
import math

math.exp( 2)


7.38905609893065

In [11]:
import math

camera_pos = np.array([5,5,5])
lookAt = np.array([0,0,0])
up = np.array([0,1,0])
look_vec = lookAt - camera_pos

w = 800
h = 800
near = 0.1
far = 100

theta_w = 45
theta_w = math.radians(theta_w)

theta_h = 45
theta_h = math.radians(theta_h)
c = - near / far


In [127]:
foreground = np.array([[3.84529972,3.84529901,3.84529877, 1]])
background = np.array([[3.26794982,3.26794863,3.26794839, 1]])

In [191]:
def get_m_view():
    look_vec = lookAt - camera_pos
    w = -look_vec / np.linalg.norm(look_vec)
    v = up - np.dot(up, w)*w
    v = v / np.linalg.norm(v)
    u = np.cross(v, w)

    m_rotate = np.array([[u[0], u[1], u[2], 0], 
                    [v[0], v[1], v[2], 0],
                    [w[0], w[1], w[2], 0],
                    [0, 0, 0, 1]])

    m_view = np.array([
        [m_rotate[0][0], m_rotate[0][1], m_rotate[0][2],  camera_pos[0]],
        [m_rotate[1][0], m_rotate[1][1], m_rotate[1][2], camera_pos[1]],
        [m_rotate[2][0], m_rotate[2][1], m_rotate[2][2], camera_pos[2]],
        [0, 0, 0, 1]
    ])

    return np.around(m_view,2)

In [192]:
def get_m_proj():
    Sxyz = np.array([[1/(far * math.tan(theta_w/2)), 0, 0, 0], 
                    [0, 1/(far * math.tan(theta_h/2)), 0, 0],
                    [0, 0, 1/far, 0],
                    [0, 0, 0, 1]])

    Mpp = np.array([[1, 0, 0, 0], 
                    [0, 1, 0, 0],
                    [0, 0, 1/(1+c),  -c/(1+c)],
                    [0, 0, -1, 0]])

    remapping = np.array([[1, 0, 0, 0], 
                    [0, 1, 0, 0],
                    [0, 0, -2,  -1],
                    [0, 0, 0, 1]])

    m_perspective = np.matmul(remapping,np.matmul(Mpp,Sxyz))
    return m_perspective


In [193]:
mat = get_m_view()
inv = np.linalg.inv(mat)

In [217]:
m_view = get_m_view()
m_proj = get_m_proj()

In [226]:
m_proj

array([[ 0.02414214,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.02414214,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.01002002, -0.002002  ],
       [ 0.        ,  0.        , -0.01      ,  0.        ]])

In [219]:
background

array([[3.26794982, 3.26794863, 3.26794839, 1.        ]])

In [228]:
background_mat = np.matmul(m_view, background.T)
background_mat = np.matmul(m_proj, background_mat)
np.around(background_mat, 2)

array([[ 0.12],
       [ 0.12],
       [-0.11],
       [-0.11]])

In [229]:
foreground_mat = np.matmul(m_view, foreground.T)
foreground_mat = np.matmul(m_proj, foreground_mat)
np.around(foreground_mat, 2)

array([[ 0.12],
       [ 0.12],
       [-0.12],
       [-0.12]])

In [13]:
camera_pos = np.array([-0.2,0,-0.2])
lookAt = np.array([0,0,0])
up = np.array([0,0,1])
look_vec = lookAt - camera_pos


In [14]:
look_vec = lookAt - camera_pos
w = look_vec / np.linalg.norm(look_vec)
w

array([0.70710678, 0.        , 0.70710678])

In [15]:
v = up - np.dot(up, w)*w
v = v / np.linalg.norm(v)
v

array([-0.70710678,  0.        ,  0.70710678])

In [16]:
u = np.cross(v, w)
u

array([ 0.,  1., -0.])

In [17]:
m_rotate = np.array([[u[0], u[1], u[2], 0], 
                    [v[0], v[1], v[2], 0],
                    [w[0], w[1], w[2], 0],
                    [0, 0, 0, 1]])
-m_rotate

array([[-0.        , -1.        ,  0.        , -0.        ],
       [ 0.70710678, -0.        , -0.70710678, -0.        ],
       [-0.70710678, -0.        , -0.70710678, -0.        ],
       [-0.        , -0.        , -0.        , -1.        ]])

In [18]:
np.transpose(m_rotate)

array([[ 0.        , -0.70710678,  0.70710678,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [-0.        ,  0.70710678,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [19]:
np.dot(np.transpose(m_rotate), np.array([0, 1.2, 1.2, 1]))

array([1.11022302e-16, 0.00000000e+00, 1.69705627e+00, 1.00000000e+00])

In [20]:
m_Translate = np.array([[1, 0, 0, -camera_pos[0]], 
                    [0, 1, 0, -camera_pos[1]],
                    [0, 0, 1, -camera_pos[2]],
                    [0, 0, 0, 1]])

In [21]:
m_view = np.around(np.matmul(m_rotate, m_Translate), 3)

In [22]:
m_view

array([[ 0.   ,  1.   ,  0.   ,  0.   ],
       [-0.707,  0.   ,  0.707,  0.   ],
       [ 0.707,  0.   ,  0.707,  0.283],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

In [23]:
inv = np.linalg.inv(m_view)
inv

array([[-0.        , -0.70721358,  0.70721358, -0.20014144],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70721358,  0.70721358, -0.20014144],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [27]:
np.log(10)

2.302585092994046

In [24]:
cam_pos = np.array([1,1,0,1])
cam_pos.dot(inv)

array([ 1.        , -0.70721358,  0.70721358,  0.79985856])

In [56]:
import math 

w = 800
h = 800
near = 0.1
far = 100

theta_w = 45
theta_w = math.radians(theta_w)

theta_h = 45
theta_h = math.radians(theta_h)

c = - near / far

Sxyz = np.array([[1/(far * math.tan(theta_w/2)), 0, 0, 0], 
                    [0, 1/(far * math.tan(theta_h/2)), 0, 0],
                    [0, 0, 1/far, 0],
                    [0, 0, 0, 1]])

Mpp = np.array([[1, 0, 0, 0], 
                    [0, 1, 0, 0],
                    [0, 0, 1/(1+c),  -c/(1+c)],
                    [0, 0, -1, 0]])

remapping = np.array([[1, 0, 0, 0], 
                    [0, 1, 0, 0],
                    [0, 0, -2,  -1],
                    [0, 0, 0, 1]])

m_perspective = np.matmul(remapping,np.matmul(Mpp,Sxyz))
m_perspective = np.around(m_perspective, 3)
m_perspective

array([[ 0.024,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.024,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.01 , -0.002],
       [ 0.   ,  0.   , -0.01 ,  0.   ]])

In [57]:
pos = np.array([3.8, 4.3, 3.7, 1])
new_p = pos.dot(m_view).dot(m_perspective)

In [124]:
new_p

array([-0.1578216,  0.0329736, -0.318393 ,  0.0024054])

In [109]:
m_perspective

array([[ 0.024,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.024,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.01 , -0.002],
       [ 0.   ,  0.   , -0.01 ,  0.   ]])

In [119]:
inv_perspective = np.linalg.inv(m_perspective)
inv_view = np.linalg.inv(m_view)

In [120]:
new_p.dot (inv_perspective).dot(inv_view)

array([3.8, 4.3, 3.7, 1. ])

In [96]:
x = 10.2

In [1]:
from sympy import *
x, y, z = symbols('x y z')

In [2]:
front_x = 0.2
front_y = 0.4

center_x = 0.21
center_y = 0.41

In [7]:
a = integrate(exp(-x), (x, 0, oo))


In [9]:
a + 0.2

1.20000000000000

In [11]:
type(a+0.2)

sympy.core.numbers.Float

In [6]:
integrate(exp((x-center_x)**2+(y-center_y)**2), (x, front_x, front_x+0.05), (y, front_y, front_y+0.05))

1.23639513938013*Integral(exp(-0.42*x)*exp(x**2), (x, 0.2, 0.25))*Integral(exp(-0.82*y)*exp(y**2), (y, 0.4, 0.45))

In [90]:
graph = np.zeros((800,800))

In [106]:
import perspective

In [107]:
a = perspective.get_m_view()
a

array([[-0.70710678,  0.        ,  0.70710678],
       [-0.40824829,  0.81649658, -0.40824829],
       [-0.57735027, -0.57735027, -0.57735027]])

In [108]:
m_view = np.array([[ a  a12  a13  0 ]
[ a21  a22  a23  0 ]
[ a31  a32  a33  0 ]
[   0    0    0  1 ]
])

SyntaxError: invalid syntax (3966108316.py, line 1)

In [134]:
import math
PI = 3.14
class Camera:
    def __init__(self, fov=60, aspect_ratio=1.0):
        # Camera parameters
        self.lookfrom = np.array([5.0, 5.0, 5.0])
        self.lookat = np.array([0.0, 0.0, 0.0])
        self.vup = np.array([0.0, 1.0, 0.0])
        self.fov = fov
        self.aspect_ratio = aspect_ratio
        
        theta = self.fov * (PI / 180.0)
        half_height = math.tan(theta / 2.0)
        half_width = self.aspect_ratio * half_height
        self.cam_origin = self.lookfrom
        
        w = self.lookfrom - self.lookat
        w = w / np.linalg.norm(w)
        u = np.cross(self.vup,w)
        u = u / np.linalg.norm(u)
        v = np.cross(w,u)
        
        self.cam_lower_left_corner = self.cam_origin - half_width * u - half_height * v - w
        self.cam_horizontal = 2 * half_width * u
        self.cam_vertical = 2 * half_height * v


    def get_ray(self, u, v):
        r = self.cam_lower_left_corner + u * self.cam_horizontal + v * self.cam_vertical - self.cam_origin
        return r
    
    def get_camera_origin(self):
        return self.cam_origin

In [135]:
my_cam = Camera()

In [137]:
my_cam.get_ray(0.2, 0.5)

array([-0.82214911, -0.57735027, -0.33255143])

In [38]:
sa.history

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.1 ,  0.  ],
       [ 0.  ,  0.2 ,  0.  ],
       [ 0.  ,  0.3 ,  0.  ],
       [ 0.  ,  0.4 ,  0.  ],
       [-0.1 ,  0.4 ,  0.  ],
       [-0.17,  0.4 ,  0.  ],
       [-0.24,  0.4 ,  0.  ],
       [-0.31,  0.4 ,  0.  ],
       [-0.38,  0.4 ,  0.  ],
       [-0.45,  0.4 ,  0.  ],
       [-0.52,  0.4 ,  0.  ],
       [-0.59,  0.4 ,  0.  ],
       [-0.66,  0.4 ,  0.  ],
       [-0.73,  0.4 ,  0.  ],
       [ 0.1 ,  0.4 ,  0.  ],
       [ 0.17,  0.4 ,  0.  ],
       [ 0.24,  0.4 ,  0.  ],
       [ 0.31,  0.4 ,  0.  ],
       [ 0.38,  0.4 ,  0.  ],
       [ 0.45,  0.4 ,  0.  ],
       [ 0.45,  0.47,  0.  ],
       [ 0.45,  0.54,  0.  ],
       [ 0.45,  0.61,  0.  ],
       [ 0.45,  0.68,  0.  ],
       [ 0.45,  0.33,  0.  ],
       [ 0.45,  0.26,  0.  ],
       [ 0.45,  0.19,  0.  ],
       [ 0.45,  0.12,  0.  ],
       [ 0.45,  0.05,  0.  ],
       [ 0.45, -0.02,  0.  ],
       [ 0.45, -0.09,  0.  ],
       [ 0.45, -0.16,  0.  ],
       [ 0

In [82]:
sa2.history

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.1 ,  0.  ,  0.  ],
       [ 0.2 ,  0.  ,  0.  ],
       [ 0.3 ,  0.  ,  0.  ],
       [ 0.4 ,  0.  ,  0.  ],
       [ 0.4 ,  0.1 ,  0.  ],
       [ 0.4 ,  0.17,  0.  ],
       [ 0.4 ,  0.24,  0.  ],
       [ 0.4 ,  0.31,  0.  ],
       [ 0.4 ,  0.38,  0.  ],
       [ 0.4 ,  0.45,  0.  ],
       [ 0.4 ,  0.52,  0.  ],
       [ 0.4 ,  0.59,  0.  ],
       [ 0.4 ,  0.66,  0.  ],
       [ 0.4 , -0.1 ,  0.  ],
       [ 0.4 , -0.17,  0.  ],
       [ 0.4 , -0.24,  0.  ],
       [ 0.4 , -0.31,  0.  ],
       [ 0.4 , -0.38,  0.  ],
       [ 0.4 , -0.45,  0.  ],
       [ 0.4 , -0.52,  0.  ],
       [ 0.47, -0.52,  0.  ],
       [ 0.54, -0.52,  0.  ],
       [ 0.61, -0.52,  0.  ],
       [ 0.68, -0.52,  0.  ],
       [ 0.75, -0.52,  0.  ],
       [ 0.75, -0.59,  0.  ],
       [ 0.75, -0.66,  0.  ],
       [ 0.75, -0.73,  0.  ],
       [ 0.75, -0.8 ,  0.  ],
       [ 0.75, -0.87,  0.  ],
       [ 0.75, -0.94,  0.  ],
       [ 0.75, -1.01,  0.  ],
       [ 0

In [83]:
def offscreen(structure, max_coordinate, min_coordinate):
    max_x = max_coordinate[0]
    max_y = max_coordinate[1]
    min_x = min_coordinate[0]
    min_y = min_coordinate[1]
    
    for i in structure.rect:
        if(i.start_x < min_x or i.start_y < min_y):
            return True
        if(i.start_x+i.scale_x > max_x or i.start_y+i.scale_y > max_y):
            return True
    
    return False

In [84]:
max_coordinate = np.array([1.0, 1.0])
min_coordinate = np.array([-1.0, -1.0])
offscreen(sa, max_coordinate, min_coordinate)

False

In [85]:
kk = []

In [86]:
kk.append(sa)

In [87]:
kk[0].rect

In [116]:
for i in sa.rect:
    i.info()
    print(i.start_x)

start_x: 0.0 start_y: 0.0 start_z: 0.0
scale_x: 0.05 scale_y: 0.45 scale_z: 0.05
 
0.0
start_x: -0.66 start_y: 0.4 start_z: 0.0
scale_x: 0.61 scale_y: 0.05 scale_z: 0.05
 
-0.66
start_x: -0.66 start_y: 0.4 start_z: 0.0
scale_x: 1.1 scale_y: 0.04 scale_z: 0.04
 
-0.66
start_x: 0.4 start_y: 0.4 start_z: 0.0
scale_x: 0.07 scale_y: 0.54 scale_z: 0.04
 
0.4
start_x: 0.1 start_y: 0.4 start_z: 0.0
scale_x: 0.47 scale_y: 0.05 scale_z: 0.05
 
0.1
start_x: 0.52 start_y: 0.47 start_z: 0.0
scale_x: 0.05 scale_y: 0.47 scale_z: 0.05
 
0.52
start_x: 0.52 start_y: 0.05 start_z: 0.0
scale_x: 0.05 scale_y: 0.33 scale_z: 0.05
 
0.52
start_x: 0.45 start_y: 0.05 start_z: 0.0
scale_x: 0.47 scale_y: 0.05 scale_z: 0.05
 
0.45
start_x: 0.59 start_y: 0.05 start_z: 0.0
scale_x: 0.4 scale_y: 0.05 scale_z: 0.05
 
0.59


In [120]:
a = 0.1
b = 0.2
c = 0.5

x = np.array([a,b,c])
np.argmax(x)

2

In [89]:
def parallel_score(history_a, history_b):
    score = 0
    parallel_pts = []
    
    for i in history_a:
        for j in history_b:
            if(i[0] == j[0] and i[1] == j[1]):
                parallel_pts.append((i[0],i[1]))
                score += 5
            elif(i[0] == j[0] or i[1] == j[1]):
                score += 1
    return score, parallel_pts

In [90]:
def check_close(struct_a):
    for i in struct_a.history:
        print(i)

In [91]:
def occlude(front_structure,position, eye):
    ray = eye - position
    ray = ray / np.linalg.norm(ray)
#     print("ray", ray)

    z_depth = front_structure.history[0][-1]
    t = (eye[-1] - z_depth) / ray[-1]
#     print("t", t)

    x = eye[0] + t*ray[0]
    y = eye[1] + t*ray[1]
    
    for i in front_structure.history:
        if x== i[0] and y == i[1]:
            return True
    
    return False

In [92]:
def occlusion_score(front_structure, points, eye):
    
    count = 0
    for i in points:
        if (occlude(front_structure, i, eye)):
            count += 1
    
    return count
        
    

In [93]:
round(random.uniform(1, 6)/100,2)

0.01

In [94]:
eye = np.array([5.0,5.0,5.0])

In [95]:
position = np.array([0.3,0.4,-0.25])

In [96]:
occlude(sa, position, eye)

False

In [97]:
class connecting_structure:
    def __init__(self, x, y, foreground_z, background_z):
        self.x = x
        self.y = y
        self.foreground_z = foreground_z
        self.background_z = background_z
    
    def get_object():
        z_start = min(self.foreground_z, self.background_z)
        z_scale = abs(self.foreground_z - self.background_z )
        
        startPos = np.array([x,y,z_start])
        scale = np.array([0.07, 0.07, z_scale])
        
        tempt = rect(startPos, scale)
        
        return tempt
    
    def get_center():
        z_start = min(self.foreground_z, self.background_z)
        z_scale = abs(self.foreground_z - self.background_z )
        
        center = np.array([self.x+0.035, self.y+0.035, z_start + 0.5*z_scale])
        
        return center

In [98]:
sa.history[0][-1]

0.0

In [99]:
a = round(int(random.random() * 5 % 5) * 0.1 + 0.2, 1)

In [100]:
b = random.random()

In [101]:
b>0.5 ? -1 : 1

SyntaxError: invalid syntax (202455723.py, line 1)

In [102]:
check_close(sa2)

[0. 0. 0.]
[0.1 0.  0. ]
[0.2 0.  0. ]
[0.3 0.  0. ]
[0.4 0.  0. ]
[0.4 0.1 0. ]
[0.4  0.17 0.  ]
[0.4  0.24 0.  ]
[0.4  0.31 0.  ]
[0.4  0.38 0.  ]
[0.4  0.45 0.  ]
[0.4  0.52 0.  ]
[0.4  0.59 0.  ]
[0.4  0.66 0.  ]
[ 0.4 -0.1  0. ]
[ 0.4  -0.17  0.  ]
[ 0.4  -0.24  0.  ]
[ 0.4  -0.31  0.  ]
[ 0.4  -0.38  0.  ]
[ 0.4  -0.45  0.  ]
[ 0.4  -0.52  0.  ]
[ 0.47 -0.52  0.  ]
[ 0.54 -0.52  0.  ]
[ 0.61 -0.52  0.  ]
[ 0.68 -0.52  0.  ]
[ 0.75 -0.52  0.  ]
[ 0.75 -0.59  0.  ]
[ 0.75 -0.66  0.  ]
[ 0.75 -0.73  0.  ]
[ 0.75 -0.8   0.  ]
[ 0.75 -0.87  0.  ]
[ 0.75 -0.94  0.  ]
[ 0.75 -1.01  0.  ]
[ 0.82 -1.01  0.  ]
[ 0.89 -1.01  0.  ]
[ 0.96 -1.01  0.  ]
[ 1.03 -1.01  0.  ]
[ 0.33 -0.52  0.  ]
[ 0.4  -0.52  0.  ]
[ 0.47 -0.52  0.  ]
[ 0.54 -0.52  0.  ]
[ 0.61 -0.52  0.  ]
[ 0.68 -0.52  0.  ]
[ 0.75 -0.52  0.  ]


In [103]:
(score, parallel_pts)= parallel_score(sa2.history, sa.history)
score

128

In [104]:
len(parallel_pts)

2

In [105]:
def block_to_rect(buffer, portion):
    start = buffer[1]
    end = buffer[-1]
    x_start = min(start[0],end[0])
    y_start = min(start[1],end[1])    
    z_start = min(start[2],end[2])
    
    x_scale = max(abs(start[0] - end[0]), 0.1 * portion)
    y_scale = max(abs(start[1] - end[1]), 0.1 * portion)
    z_scale = max(abs(start[2] - end[2]), 0.1 * portion)
    
    if(x_scale > 0.1 * portion):
        x_scale += 0.1 * portion
        
    if(y_scale > 0.1 * portion):
        y_scale += 0.1 * portion
        
    if(z_scale > 0.1 * portion):
        z_scale += 0.1 * portion

    startPos = np.array([x_start,y_start,z_start])
    scale = np.array([x_scale,y_scale,z_scale])
    
    tempt = rect(startPos, scale)
    return tempt


In [106]:
class rect:
    def __init__(self, startPos, scale): 
        self.start_x = round(startPos[0],2);
        self.start_y = round(startPos[1],2);
        self.start_z = round(startPos[2],2);
        
        self.scale_x = round(scale[0],2);
        self.scale_y = round(scale[1],2);
        self.scale_z = round(scale[2],2);
    
    def info(self):
        print("start_x:",self.start_x,"start_y:",self.start_y,"start_z:",self.start_z)
        print("scale_x:",self.scale_x,"scale_y:",self.scale_y,"scale_z:",self.scale_z)
        print(" ")
    
    def center_x(self):
        return round(self.start_x + 0.5 * self.scale_x, 1)
    
    def center_y(self):
        return round(self.start_y + 0.5 * self.scale_y,1)

In [107]:
b1 = np.array([[0,0,0], [0,1,0],[0,2,0],[0,3,0], [0,4,0]])
b2 = np.array([[0,0,1], [0,1,1],[0,2,1],[0,3,1], [0,4,1]])

In [108]:
c = np.append(b1, b2, axis=0)
c

array([[0, 0, 0],
       [0, 1, 0],
       [0, 2, 0],
       [0, 3, 0],
       [0, 4, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 2, 1],
       [0, 3, 1],
       [0, 4, 1]])

In [109]:
rect_list = []

In [110]:
rect_list.append(block_to_rect(b1))
rect_list.append(block_to_rect(b2))
rect_list

TypeError: block_to_rect() missing 1 required positional argument: 'portion'

In [111]:
pos = np.array([2.0,2.0,4.0])
np.array([[0.0,0.0,0.0]])

array([[0., 0., 0.]])

In [112]:
len(sa.next_possibles)

6

In [113]:
import random
def rand_index(input_array):
    return int(random.random()* len(input_array) % len(input_array))

In [114]:
for i in range(20):
    print(rand_index(sa.next_possibles))

3
1
4
2
0
4
0
4
1
3
2
2
5
1
4
1
4
2
0
2


In [38]:
pos[0]

2.0

In [45]:
def gen_seed(pos):
    seed = pos
    for i in range(1,5):
        tempt = np.array([pos[0]+0.1*i,pos[1],pos[2]])
        seed = np.vstack([seed, tempt])
    return seed

In [46]:
a=gen_seed(pos)

In [56]:
a[-1][0]

2.4

In [51]:
def get_next_possible(pos):
    tempt1 = np.array([pos[0],pos[1]+0.1,pos[2],4])
    tempt2 = np.array([pos[0],pos[1]-0.1,pos[2],3])
    
    return np.vstack([tempt1, tempt2])

In [52]:
pos_b = a[-1]

In [53]:
x = get_next_possible(pos_b)
x

array([[2.4, 2.1, 4. , 4. ],
       [2.4, 1.9, 4. , 3. ]])

In [218]:
def too_close(structure):
    horizontal = []
    vertical = []

    for i in structure.rect:
        if(i.scale_x > i.scale_y):
            horizontal.append(i)
        else:
            vertical.append(i)
    
    for i in horizontal:
        for j in horizontal:
            if(i.start_y != j.start_y and abs(i.start_y - j.start_y)<0.2):
                return True
    
    for i in vertical:
        for j in vertical:
            if(i.start_x != j.start_x and abs(i.start_x - j.start_x)<0.2):
                return True
    
    return False

In [219]:
too_close(sa)

True

In [210]:
sa.rect

In [223]:
import openai

response = openai.Image.create(
  prompt="a white siamese cat",
  n=1,
  size="1024x1024"
)
image_url = response['data'][0]['url']


AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.

In [74]:
s = [2, 3, 1, 4, 5, 3]
a = sorted(range(len(s)), key=lambda k: s[k])
a[:2]

[2, 0]

In [3]:
import numpy as np
import random

In [16]:
int(np.random.normal(loc=10.0, scale=5.0, size=None))


20